### 라이브러리 설치

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install webdriver-manager
!pip install selenium
!pip install pandas
!pip install tqdm

### 라이브러리 임포트

In [ ]:
#라이브러리 임포트
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm.auto import tqdm 

### 필요한 함수 정의

In [3]:
## 뉴스 제목 크롤링 하는 함수 정의
def get_news_title(tag):
    try:
        title = tag.parent.parent.parent.find('a', 'news_tit').get('title')
        
    except:
        title = tag.parent.parent.find('a', 'elss sub_tit').get('title')
        
    return title

In [4]:
## 뉴스 링크에 접속해서 컨텐츠 크롤링하는 함수 정의 
def find_content(soup):
    try:
        content = soup.find('article').text
    except:
        content = soup.find('div','_article_content').text
    return content

### 크롤링

In [ ]:
# 자신의 드라이버 경로 지정하기
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = s)
time.sleep(2)

# 윈도우 크기 조절
driver.maximize_window()

In [ ]:
#데이터 범위 (날짜) 지정
date_range = list(map(lambda x: x.strftime('%Y.%m.%d'), pd.date_range(start = '20240422', end = '20240430')))

In [ ]:
## 데이터 긁어와서 df에 저장 
df = pd.DataFrame()
query = '민희진'

for date in date_range:
    url = f'https://search.naver.com/search.naver?where=news&query={query}&ds={date}&de={date}'
    # 스크롤을 페이지 끝까지 내리는 코드
    driver.get(url)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    n = 0
    start = time.time()
    while n < 100: # 최대 5번 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # 스크롤이 끝까지 내려가기를 기다리는 시간
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height: # 더 이상 내려가지 않는다는 의미
            break

        last_height = new_height
        n += 1

        if n % 100 == 0:
            end = time.time()
            
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    title_list = list(map(get_news_title, soup.find_all('a', text = '네이버뉴스')))
    url_list = list(map(lambda x: x.get('href'), soup.find_all('a', text = '네이버뉴스')))
    
    part_df = pd.DataFrame({'title' : title_list,
                              'url' : url_list,
                              'date' : date})
    
    df = pd.concat([df, part_df])

In [ ]:
content_list = []
entertain_url = []
for url in tqdm(df['url']):
    
    if 'm.entertain.naver.com' not in url:
        r = requests.get(url)
        time.sleep(1)

        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        content = soup.find('article')
        if content.text is not None:
            content = soup.find('article').text
        else:
            content = ''
        content_list.append(content)
       
        
    else:
        entertain_url.append(url)
        content = ''
        content_list.append(content)

In [ ]:
df['content'] = content_list

In [ ]:
driver.quit()